<div class="alert alert-danger" style="color:black"><b>Running ML-LV Jupyter Notebooks:</b><br>
    <ol>
        <li>Make sure you are running all notebooks using the <code>adv_ai</code> kernel.
        <li><b>It is very important that you do not create any additional files within the weekly folders on CSCT cloud.</b> Any additional files, or editing the notebooks with a different environment may prevent submission/marking of your work.</li>
            <ul>
                <li>NBGrader will automatically fetch and create the correct folders files for you.</li>
                <li>All files that are not the Jupyter notebooks should be stored in the 'ML-LV/data' directory.</li>
            </ul>
        <li>Please <b>do not pip install</b> any python packages (or anything else). You should not need to install anything to complete these notebooks other than the packages provided in the Jupyter CSCT Cloud environment.</li>
    </ol>
    <b>If you would like to run this notebook locally you should:</b><br>
    <ol>
        <li>Create an environment using the requirements.txt file provided. <b>Any additional packages you install will not be accessible when uploaded to the server and may prevent marking.</b></li>
        <li>Download a copy  of the notebook to your own machine. You can then edit the cells as you wish and then go back and copy the code into/edit the ones on the CSCT cloud in-place.</li>
        <li><b>It is very important that you do not re-upload any notebooks that you have edited locally.</b> This is because NBGrader uses cell metadata to track marked tasks. <b>If you change this format it may prevent marking.</b></li>
    </ol>
</div>

# Practical 5: Language Modelling

Language models attempt to create a generalised long-term model of language. That is, a model that has learned the characteristics of the language it was trained on, such as grammar, punctuation, the relationship between words and so on. Typically, language models predict the next token of a sequence, given the previous tokens, or simply calculate the probability of a sequence of tokens. This can be implemented with a simple n-gram language model, but these are limited to predicting only sequences of n-grams (word combinations) observed within the training data.

In contrast, ML language models are much more robust and capable of generating more varied and 'imaginative' texts. RNN are well suited to this task due to their sequential processing of input. However, modern stat-of-the-art language models, such as [GPT-4](https://openai.com/gpt-4), use [Transformers](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)). In either case LM are typically trained on very large natural language datasets, such as Wikipedia, Amazon reviews, Google books, or in our case Sherlock Holmes stories! Importantly, and similar to word vectors, once trained, the model of language that has been learned can be transferred to other downstream tasks, such as classification and translation.

In the first part of this practical we will create two n-gram language models. The first will simply calculate the probability of an input sentence, given the training corpus. The second will generate the next word, given the previous words in a sequence.

In the second part of this practical we will create an RNN language model using the complete works of [Sherlock Holmes by Sir Aurthur Conan Doyle](https://sherlock-holm.es/) as a training corpus. Then we will use the trained model to generate sentences in the style of Conan Doyle, and explore various different methods of sampling from the word probabilities generated by the model.

The objectives of this practical are:
1. Understand the key concepts of language modelling and evaluation: calculating the probability of a sentence or next token and calculating perplexity

2. Introduce new ML techniques that are useful for building large models: data generators, custom metrics, early stopping and checkpointing

3. Compare and contrast different sampling methods, such as greedy, temperature and top-k

# 1 N-gram Language Models

## 1.0 Import libraries

In [2]:
import re
import numpy as np
from collections import Counter
from nltk import ngrams

## 1.1 Probability of a sentence

We can create simple language models using n-grams. Whether we are calculating the probability of a sequence, or trying to predict the next word, the first step is the same: calculate the probability of a word given the previous word (bi-gram), or words (tri-gram +).

So, for each n-gram within the training corpus we need to calculate:

$P(word|previous \; words) = \frac{count(previous \; words, \; word)}{count(previous \; words)}$

The language model is then effectively the set of n-gram probabilites that have been calculated.

In [3]:
# Define the corpus
corpus = ['This is a dog', 'This is a cat', 'I love my cat', 'I love my dog', 'This is my name']

# Count the unigrams and n-grams
N = 2
n_grams = []
uni_grams = []
for sent in corpus:
    tokens = sent.lower().split()
    n_grams.extend(list(ngrams(tokens, N)))
    uni_grams.extend(tokens)

n_gram_count = Counter(n_grams)
print(f"{N}-grams: {n_gram_count}")
uni_gram_count = Counter(uni_grams)
print(f"Uni-grams: {uni_gram_count}")

# Calculate the n-gram probabilities
n_gram_probs = {}
for n_gram in n_grams:
    n_gram_probs[n_gram] = n_gram_count[n_gram] / uni_gram_count[n_gram[0]]
    print(f"P({n_gram[1:]}|{n_gram[0]}) = {n_gram_probs[n_gram]:.4f}")

2-grams: Counter({('this', 'is'): 3, ('is', 'a'): 2, ('i', 'love'): 2, ('love', 'my'): 2, ('a', 'dog'): 1, ('a', 'cat'): 1, ('my', 'cat'): 1, ('my', 'dog'): 1, ('is', 'my'): 1, ('my', 'name'): 1})
Uni-grams: Counter({'this': 3, 'is': 3, 'my': 3, 'a': 2, 'dog': 2, 'cat': 2, 'i': 2, 'love': 2, 'name': 1})
P(('is',)|this) = 1.0000
P(('a',)|is) = 0.6667
P(('dog',)|a) = 0.5000
P(('is',)|this) = 1.0000
P(('a',)|is) = 0.6667
P(('cat',)|a) = 0.5000
P(('love',)|i) = 1.0000
P(('my',)|love) = 1.0000
P(('cat',)|my) = 0.3333
P(('love',)|i) = 1.0000
P(('my',)|love) = 1.0000
P(('dog',)|my) = 0.3333
P(('is',)|this) = 1.0000
P(('my',)|is) = 0.3333
P(('name',)|my) = 0.3333


Once we have calculated the n-gram probabilities, calculating the probability of a sentence is simply:

1. Count the n-grams in the input sentence

2. Find the product of all n-grams that exist within both the model and the input sentence

In [4]:
# Sentence to calculate probability of
sentence = "This is my dog"

# Count n-grams
sentence_n_grams = list(ngrams([token.lower() for token in sentence.split()], N))
print(f"{N}-grams: {sentence_n_grams}")

# Calculate probability of sentence
probability = 1
# For each n-gram in sentence
for n_gram in sentence_n_grams:

    # If its probability is in the training corpus
    if n_gram in n_gram_probs:
        print(f"{N}-gram: {n_gram} probability: {round(n_gram_probs[n_gram], 3)}")

        # Multiply the probability of the n-gram to the total probability
        probability *= n_gram_probs[n_gram]

print(f"Probablility of sentence \"{sentence}\" = {probability:.4f}")

2-grams: [('this', 'is'), ('is', 'my'), ('my', 'dog')]
2-gram: ('this', 'is') probability: 1.0
2-gram: ('is', 'my') probability: 0.333
2-gram: ('my', 'dog') probability: 0.333
Probablility of sentence "This is my dog" = 0.1111


## 1.2 Probability of next word

To calculate the probability of the next word in a sequence we need to build a language model in the same manner as before, by finding the probability of a word given the previous word(s):

$P(word|previous \; words) = \frac{count(previous \; words, \; word)}{count(previous \; words)}$

Note that here we have added two special tokens, `<s>` signifies the start of a sentence and `</s>` the end. This is often useful when generating text because it provides a generic starting, and potentially end point, for the generation process. Otherwise you would always need to provide a starting token/word, such as 'I', and this would influence selection of the following tokens.

In [5]:
# Define the corpus
corpus = ['<s> I am Sam </s>', '<s> Sam I am </s>', '<s> I do not like green eggs and ham </s>']

# Calculate the unigram and n-gram counts
N = 2
n_grams = []
uni_grams = []
for sent in corpus:
    tokens = sent.lower().split()
    n_grams.extend(list(ngrams(tokens, N)))
    uni_grams.extend(tokens)

n_gram_count = Counter(n_grams)
print(f"{N}-grams: {n_gram_count}")
uni_gram_count = Counter(uni_grams)
print(f"Uni-grams: {uni_gram_count}")

# Calculate the n-gram probabilities
n_gram_probs = {}
for n_gram in n_grams:
    n_gram_probs[n_gram] = n_gram_count[n_gram] / uni_gram_count[n_gram[0]]
    print(f"P({n_gram[1:]}|{n_gram[0]}) = {n_gram_probs[n_gram]:.4f}")

2-grams: Counter({('<s>', 'i'): 2, ('i', 'am'): 2, ('am', 'sam'): 1, ('sam', '</s>'): 1, ('<s>', 'sam'): 1, ('sam', 'i'): 1, ('am', '</s>'): 1, ('i', 'do'): 1, ('do', 'not'): 1, ('not', 'like'): 1, ('like', 'green'): 1, ('green', 'eggs'): 1, ('eggs', 'and'): 1, ('and', 'ham'): 1, ('ham', '</s>'): 1})
Uni-grams: Counter({'<s>': 3, 'i': 3, '</s>': 3, 'am': 2, 'sam': 2, 'do': 1, 'not': 1, 'like': 1, 'green': 1, 'eggs': 1, 'and': 1, 'ham': 1})
P(('i',)|<s>) = 0.6667
P(('am',)|i) = 0.6667
P(('sam',)|am) = 0.5000
P(('</s>',)|sam) = 0.5000
P(('sam',)|<s>) = 0.3333
P(('i',)|sam) = 0.5000
P(('am',)|i) = 0.6667
P(('</s>',)|am) = 0.5000
P(('i',)|<s>) = 0.6667
P(('do',)|i) = 0.3333
P(('not',)|do) = 1.0000
P(('like',)|not) = 1.0000
P(('green',)|like) = 1.0000
P(('eggs',)|green) = 1.0000
P(('and',)|eggs) = 1.0000
P(('ham',)|and) = 1.0000
P(('</s>',)|ham) = 1.0000


Once we have created the model, by calculating the n-gram probabilities, we can use it to generate text:

1. First start with the seed tokens/text which will 'prompt' the model for the next token. In this case we can simply use the start of a sentence token (`<s>`).

2. Loop until the end of sentence token (`</s>`) is generated, or a maximum sequence length is reached. At each step:

    1. Find *all* the possible next tokens given the previous token.

    2. Select the next token using the greedy sampling method.

    3. Add the selected token to the generated text.


In [6]:
# Set the seed text
generated_text = "<s>"
# Variable to hold the next token
next_token = ''
# Set the maximum sequence length
max_seq_len = 10

# While next token is not end of sentence and max sequence length is not reached
i = 0
while next_token != '</s>' and len(generated_text.split()) < max_seq_len:

    # Tokenize generated text
    tokens = generated_text.lower().split()

    possible_next_tokens = {}
    # Find possible next tokens
    for n_gram, prob in n_gram_probs.items():
        # If last token of generated text is the first token of n-gram
        if n_gram[0] == tokens[-1]:
            # Add n-gram to possible next tokens
            possible_next_tokens[n_gram[1]] = prob

    print(f'Possible tokens: {[f"{k}: {v:.4f}" for k, v in possible_next_tokens.items()]}')

    # Greedily select next token

    # Find highest probability
    highest_prob = list(sorted(possible_next_tokens.items(), key=lambda item: item[1]))[-1][1]

    # Remove all tokens with lower probability (in case of ties)
    possible_next_tokens = {token: prob for token, prob in possible_next_tokens.items() if prob == highest_prob}
    
    # Randomly select next token
    next_token = np.random.choice(list(possible_next_tokens.keys()))

    print(f'Next token: {next_token}\n')

    # Add next token to generated text
    generated_text += ' ' + next_token
    i += 1

print(f'Generated text: {generated_text}')

Possible tokens: ['i: 0.6667', 'sam: 0.3333']
Next token: i

Possible tokens: ['am: 0.6667', 'do: 0.3333']
Next token: am

Possible tokens: ['sam: 0.5000', '</s>: 0.5000']
Next token: sam

Possible tokens: ['</s>: 0.5000', 'i: 0.5000']
Next token: </s>

Generated text: <s> i am sam </s>


<div class="alert alert-info" style="color:black"><h2>1.3 Exercise: Different sampling methods</h2>

The above algorithm uses a 'greedy' sampling method, which simply selects the most probable token from all possible next tokens. However, even with large sophisticated language models greedy sampling often results in generic and repetitive text.

1. In the following cell complete the `generate_text()` function by adding a few other sampling methods.  It should take the following arguments and return a string of generated text:
    - `model` is an n-gram language model consisting of a dictionary of words (keys) and probabilities (values).
    - `seed_text` is the seed text to start generation with. Typically the `<s>` token.
    - `max_seq_len` is the maximum possible length of a generated sequence.
    - `sampling_method` is the sampling method to use. One of 'greedy', 'eager', 'random' or 'weighted_random'.
    - `verbose` is whether to print the generated tokens at each generation step.

2. Add the following sampling methods:

    - 'eager', which selects the first possible token

    - 'random', which randomly selects from all possible tokens
    
    - 'weighted_random', which randomly selects from all possible tokens weighted by the token probability. This can be simply implemented using `np.random.choice()`

3. You can test the function using the Green Eggs and Ham or I Wandered Lonely as a Cloud poem by William Wordsworth (below).

<br>
<b>This exercise is <u>not</u> marked.</b>
</div>

In [14]:
def generate_text(model, seed_text='<s>', max_seq_len=10, sampling_method='greedy', verbose=True):
    """Generates text by sampling from an n-gram language model.
    
    Arguments:
        model (dict): An n-gram language model as a dictionary of n-grams and their probabilities
        seed_text (str): The seed text to start the generation with
        max_seq_len (int): The maximum length of the generated sequence
        sampling_method (str): The sampling method to use. One of 'greedy', 'eager', 'random' or 'weighted_random'
        verbose (bool): Whether to print the generated tokens at each generation step
    """

    # Set the seed text
    generated_text = seed_text
    # Variable to hold the next token
    next_token = ''

    # While next token is not end of sentence and max sequence length is not reached
    i = 0
    while next_token != '</s>' and len(generated_text.split()) < max_seq_len:

        # Tokenize generated text
        tokens = generated_text.lower().split()

        possible_next_tokens = {}
        # Find possible next tokens
        for n_gram, prob in model.items():
            # If last token of generated text is the first token of n-gram
            if n_gram[0] == tokens[-1]:
                # Add n-gram to possible next tokens
                possible_next_tokens[n_gram[1]] = prob

        if verbose:
            print(f'Possible tokens: {[f"{k}: {v:.4f}" for k, v in possible_next_tokens.items()]}')

        # Select next token
        # Greedy choice (highest probability)
        if sampling_method == 'greedy':
            # Find highest probability
            highest_prob = list(sorted(possible_next_tokens.items(), key=lambda item: item[1]))[-1][1]
            # Remove all tokens with lower probability (in case of ties)
            possible_next_tokens = {token: prob for token, prob in possible_next_tokens.items() if prob == highest_prob}
            # Randomly select next token
            next_token = np.random.choice(list(possible_next_tokens.keys()))

        # YOUR CODE HERE
        # raise NotImplementedError()
        # first token found
        elif sampling_method == 'eager':
            if possible_next_tokens:
                # Take the first token from the dictionary
                next_token = list(possible_next_tokens.keys())[0]
            else:
                # If no possible next tokens, end generation
                next_token = '</s>'

        # Random choice (uniform random selection)
        elif sampling_method == 'random':
            if possible_next_tokens:
                # Randomly select a token with equal probability
                next_token = np.random.choice(list(possible_next_tokens.keys()))
            else:
                # If no possible next tokens, end generation
                next_token = '</s>'

        # Weighted random choice (probability-weighted selection)
        elif sampling_method == 'weighted_random':
            if possible_next_tokens:
                tokens = list(possible_next_tokens.keys())
                probs = list(possible_next_tokens.values())
                # Normalize probabilities to sum to 1
                probs = np.array(probs) / sum(probs)
                # Select token weighted by probabilities
                next_token = np.random.choice(tokens, p=probs)
            else:
                # If no possible next tokens, end generation
                next_token = '</s>'

        else:
            raise ValueError(f"Unknown sampling method: {sampling_method}")
        
        if verbose:
            print(f'Next token: {next_token}\n')

        # Add next token to generated text
        generated_text += ' ' + next_token
        i += 1
    return generated_text

In [11]:
corpus = ["I wandered lonely as a cloud",
"That floats on high o'er vales and hills,",
"When all at once I saw a crowd,",
"A host, of golden daffodils;",
"Beside the lake, beneath the trees,",
"Fluttering and dancing in the breeze.",

"Continuous as the stars that shine",
"And twinkle on the milky way,",
"They stretched in never-ending line",
"Along the margin of a bay:",
"Ten thousand saw I at a glance,",
"Tossing their heads in sprightly dance.",

"The waves beside them danced; but they",
"Out-did the sparkling waves in glee:",
"A poet could not but be gay,",
"In such a jocund company:",
"I gazed—and gazed—but little thought",
"What wealth the show to me had brought:",

"For oft, when on my couch I lie",
"In vacant or in pensive mood,",
"They flash upon that inward eye",
"Which is the bliss of solitude;",
"And then my heart with pleasure fills,",
"And dances with the daffodils"]

# Preprocess the corpus
for i, sent in enumerate(corpus):
    # Remove punctuation
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # Add start and end tokens and strip whitespace
    corpus[i] = '<s> ' + sent.strip() + ' </s>'

# Calculate the unigram and n-gram counts
N = 2
n_grams = []
uni_grams = []
for sent in corpus:
    tokens = sent.lower().split()
    n_grams.extend(list(ngrams(tokens, N)))
    uni_grams.extend(tokens)

n_gram_count = Counter(n_grams)
print(f"{N}-grams: {n_gram_count}")
uni_gram_count = Counter(uni_grams)
print(f"Uni-grams: {uni_gram_count}")

# Calculate the n-gram probabilities
wordsworth_n_gram_probs = {}
for n_gram in n_grams:
    wordsworth_n_gram_probs[n_gram] = n_gram_count[n_gram] / uni_gram_count[n_gram[0]]
    print(f"P({n_gram[1:]}|{n_gram[0]}) = {wordsworth_n_gram_probs[n_gram]:.4f}")

2-grams: Counter({('<s>', 'and'): 3, ('<s>', 'i'): 2, ('<s>', 'a'): 2, ('daffodils', '</s>'): 2, ('<s>', 'they'): 2, ('<s>', 'in'): 2, ('i', 'wandered'): 1, ('wandered', 'lonely'): 1, ('lonely', 'as'): 1, ('as', 'a'): 1, ('a', 'cloud'): 1, ('cloud', '</s>'): 1, ('<s>', 'that'): 1, ('that', 'floats'): 1, ('floats', 'on'): 1, ('on', 'high'): 1, ('high', 'o'): 1, ('o', 'er'): 1, ('er', 'vales'): 1, ('vales', 'and'): 1, ('and', 'hills'): 1, ('hills', '</s>'): 1, ('<s>', 'when'): 1, ('when', 'all'): 1, ('all', 'at'): 1, ('at', 'once'): 1, ('once', 'i'): 1, ('i', 'saw'): 1, ('saw', 'a'): 1, ('a', 'crowd'): 1, ('crowd', '</s>'): 1, ('a', 'host'): 1, ('host', 'of'): 1, ('of', 'golden'): 1, ('golden', 'daffodils'): 1, ('<s>', 'beside'): 1, ('beside', 'the'): 1, ('the', 'lake'): 1, ('lake', 'beneath'): 1, ('beneath', 'the'): 1, ('the', 'trees'): 1, ('trees', '</s>'): 1, ('<s>', 'fluttering'): 1, ('fluttering', 'and'): 1, ('and', 'dancing'): 1, ('dancing', 'in'): 1, ('in', 'the'): 1, ('the', 'bre

In [12]:
# Call the function to generate text
generated_text = generate_text(wordsworth_n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='greedy')
print(f'Generated text: {generated_text}')

Possible tokens: ['i: 0.0833', 'that: 0.0417', 'when: 0.0417', 'a: 0.0833', 'beside: 0.0417', 'fluttering: 0.0417', 'continuous: 0.0417', 'and: 0.1250', 'they: 0.0833', 'along: 0.0417', 'ten: 0.0417', 'tossing: 0.0417', 'the: 0.0417', 'out: 0.0417', 'in: 0.0833', 'what: 0.0417', 'for: 0.0417', 'which: 0.0417']
Next token: and

Possible tokens: ['hills: 0.1667', 'dancing: 0.1667', 'twinkle: 0.1667', 'gazed: 0.1667', 'then: 0.1667', 'dances: 0.1667']
Next token: dances

Possible tokens: ['with: 1.0000']
Next token: with

Possible tokens: ['pleasure: 0.5000', 'the: 0.5000']
Next token: pleasure

Possible tokens: ['fills: 1.0000']
Next token: fills

Possible tokens: ['</s>: 1.0000']
Next token: </s>

Generated text: <s> and dances with pleasure fills </s>


In [13]:
np.random.seed(42)

# Test the eager sampling method
generated_text = generate_text(n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='eager', verbose=False)
assert generated_text == '<s> i am sam </s>', f'Generated text: {generated_text}'

# Test the random sampling method
generated_text = generate_text(n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='random', verbose=False)
assert generated_text == '<s> i do not like green eggs and ham </s>', f'Generated text: {generated_text}'

# Test the weighted random sampling method
generated_text = generate_text(n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='weighted_random', verbose=False)
assert generated_text == '<s> sam i am sam </s>', f'Generated text: {generated_text}'

# Test the eager sampling method
generated_text = generate_text(wordsworth_n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='eager', verbose=False)
assert generated_text == '<s> i wandered lonely as a cloud </s>', f'Generated text: {generated_text}'

# Test the random sampling method
generated_text = generate_text(wordsworth_n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='random', verbose=False)
assert generated_text == '<s> ten thousand saw a glance </s>', f'Generated text: {generated_text}'

# Test the weighted random sampling method
generated_text = generate_text(wordsworth_n_gram_probs, seed_text='<s>', max_seq_len=10, sampling_method='weighted_random', verbose=False)
assert generated_text == '<s> along the lake beneath the breeze </s>', f'Generated text: {generated_text}'

print('All tests passed!')

All tests passed!


<div class="alert alert-success" style="color:black"><h3>Before you submit this notebook to NBGrader for marking:</h3> 

1. Make sure have completed all exercises marked by <span style="color:blue">**blue cells**</span>.
2. For automatically marked exercises ensure you have completed any cells with `# YOUR CODE HERE`. Then click 'Validate' button above, or ensure all cells run without producing an error.
3. For manually marked exercises ensure you have completed any cells with `"YOUR ANSWER HERE"`.
4. Ensure all cells are run with their output visible.
5. Fill in your student ID (**only**) below.
6. You should now **save and download** your work.

</div>

**Student ID:** 15006280